In [1]:
from route import Route
from route_api import RouteAPI

current_position = {'longitude': 132.328816,
                    'latitude': -14.482998}
final_position = {'longitude': 133.8750197,
                  'latitude': -23.7090616}
delta_spacing = 2000.0 # in meters
number_sites = 150

route = Route()
api_route = RouteAPI(route)

api_route_df = api_route.cut_route_data(current_position, number_sites=number_sites, print_is_requested=True)
# api_route_df = api_route.cut_route_data(current_position, number_sites=number_sites, delta_spacing=delta_spacing, print_is_requested=True)
# api_route_df = api_route.cut_route_data(current_position=current_position, final_position=final_position, number_sites=number_sites, print_is_requested=False)
# route.find_closest_row(current_position)

# route.insert_to_control_stops()

Nearest index in csv file: 1706
Nearest index in dataframe: 1704
      longitude   latitude  altitudeSmoothed  inclinationSmoothed  \
0    132.328816 -14.482998        116.313963             0.002361   
1    132.451862 -14.554011        167.520724             0.004404   
2    132.601405 -14.625809        179.912251             0.005640   
3    132.744195 -14.698370        180.053066            -0.005265   
4    132.885540 -14.776446        153.559783             0.001028   
..          ...        ...               ...                  ...   
145  138.293359 -34.363255         13.498104            -0.000080   
146  138.400203 -34.497804         15.831335            -0.007119   
147  138.522740 -34.622923         15.585138            -0.000341   
148  138.594670 -34.771615         14.457534            -0.008603   
149  138.600712 -34.928189         64.449372            -0.011632   

      cumDistance  maxSpeed       theta  timeAtMaxSpeed  cumTimeAtMaxSpeed  \
0    3.233247e+05        80 

In [4]:
from Solcast_executer import SolcastExecuter
solcast_executer = SolcastExecuter()

# TODO TIMEOUT ERROR BUG
# current_position = {'longitude': 130.9064683,
#                     'latitude': -12.4479361}
# raw_df = solcast_executer.get_forecast(current_position)
raw_df = solcast_executer.get_forecasts(api_route_df, hours_in_advance=48)
print(raw_df)

                                        tt   gh     ff   dd    rh   rr
cumDistance  time                                                     
3.233247e+05 2023-10-16 16:00:00+09:30  38  639  12.24  118   3.9  0.0
             2023-10-16 16:15:00+09:30  38  579  12.24  116   3.9  0.0
             2023-10-16 16:30:00+09:30  38  517  12.24  113   3.9  0.0
             2023-10-16 16:45:00+09:30  38  453  12.60  111   4.0  0.0
             2023-10-16 17:00:00+09:30  38  386  12.60  109   4.1  0.0
...                                     ..  ...    ...  ...   ...  ...
3.026918e+06 2023-10-18 15:00:00+09:30  23  718  11.16  227  26.9  0.0
             2023-10-18 15:15:00+09:30  23  673  11.16  224  27.1  0.0
             2023-10-18 15:30:00+09:30  23  625  11.16  221  27.3  0.0
             2023-10-18 15:45:00+09:30  23  574  11.16  219  27.5  0.0
             2023-10-18 16:00:00+09:30  23  522  11.16  216  27.7  0.0

[28950 rows x 6 columns]


In [5]:
from Meteotest_executer import MeteotestExecuter
from Meteotest_requester import MeteotestRequester
from Meteotest_parser import MeteotestParser

api_parser = MeteotestParser()
api_requester = MeteotestRequester(parser=api_parser, print_is_requested=False)
api_executer = MeteotestExecuter(requester=api_requester)

Response status from siteinfo: OK.


In [ ]:
# api_requester.get_site_info(print_is_requested=True)
# api_executer.get_all_site_id

In [ ]:
# api_requester.get_site_add(name="0", position=current_position, print_is_requested=True)
# api_executer.add_sites(api_route_df, print_is_requested=True)

In [ ]:
# api_requester.get_site_delete(585394, print_is_requested=True)
# api_executer.delete_all_sites()

In [ ]:
# site_id = 54689
# name = "404"
# position = {'latitude': 136,
#             'longitude': -30}

# api_requester.get_site_edit(site_id, name=name, position=position)

# api_route_df.index=api_executer.get_all_site_id ########TODO ADAPT DIRECTLY 
# api_executer.edit_sites(api_route_df)

In [ ]:
# raw_df = api_requester.get_solar_forecast()
# print(raw_df.head())

# raw_df = api_requester.get_solar_forecast_cloudmove()
# print(raw_df)
# raw_SF_df, _, raw_CM_df, _ = api_executer.get_new_forecasts()

# print(raw_SF_df.head(), raw_CM_df.head())

In [6]:
sites_df = api_requester.get_current_sites

from preprocessor import Preprocessor
preprocessor = Preprocessor()
hours_in_advance = None
preprocessed_df, preprocessed_df = preprocessor.preprocess(api_route_df, sites_df, raw_df, hours_in_advance)
preprocessor.save_data2folder()
print(preprocessed_df)

#TODO CHECK WIND DECOMPOSITION: ERROR SEEN IN PLOTS

Data saved to G:/Drive condivisi/AlphaCentauri/SolarCar_22 23/6. Strategy & Simulation/ss_online_data/Forecast\20231016_155337_SC.
                                        temperature  globalIrradiance  \
cumDistance  time                                                       
3.233247e+05 2023-10-16 16:00:00+09:30           41               639   
             2023-10-16 16:15:00+09:30           41               579   
             2023-10-16 16:30:00+09:30           41               517   
             2023-10-16 16:45:00+09:30           41               453   
             2023-10-16 17:00:00+09:30           38               386   
...                                             ...               ...   
3.026918e+06 2023-10-18 15:00:00+09:30           26               718   
             2023-10-18 15:15:00+09:30           26               673   
             2023-10-18 15:30:00+09:30           26               625   
             2023-10-18 15:45:00+09:30           26               

In [ ]:
# from db_query import DbQuerier

# db_querier = DbQuerier()

# v = db_querier.query_velocity()
# soc = db_querier.query_soc()
# print(v, soc)

# db_querier.save_data2folder()

In [7]:
from opt_reader import OptReader

opt_reader = OptReader()
DP_data = opt_reader.get_optimal_data

Data read from C:/Users/giaco/Git_Repositories/aCentauri/ss_dp/Saved_Files/Results_DP_Test.csv.


In [ ]:
# import folium

# # Assuming you have these dataframes already
# control_stops_df = route.get_control_stops_data
# sites_df = api_requester.get_current_sites
# rt = route.get_route_data

# # Define the bounds
# min_lng = rt['longitude'].min()
# min_lat = rt['latitude'].min()
# max_lng = rt['longitude'].max()
# max_lat = rt['latitude'].max()
# bounds = [[max_lat, min_lng], [min_lat, max_lng]]

# # Create a base map centered around Australia
# middle_lat = (max_lat - min_lat) / 2
# middle_lng = (max_lng - min_lng) / 2
# map = folium.Map(location=[middle_lat, middle_lng],
#                  zoom_start=0)

# # Set the map boundaries to show only Australia
# map.fit_bounds(bounds)

# # Add the route lines
# folium.PolyLine(
#     locations=rt[['latitude', 'longitude']].values.tolist(),
#     color="green",
#     weight=4,
#     opacity=1
# ).add_to(map)

# # Add api sites
# for _, row in sites_df.iterrows():
#     folium.CircleMarker(
#         location=[row['latitude'], row['longitude']],
#         radius=2,
#         color="blue",
#         fill=True,
#         fill_color="blue",
#         fill_opacity=1,
#         tooltip=row['name']
#     ).add_to(map)

# # Add control stops
# for _, row in control_stops_df.iterrows():
#     folium.CircleMarker(
#         location=[row['latitude'], row['longitude']],
#         radius=3,
#         color="orange",
#         fill=True,
#         fill_color="orange",
#         fill_opacity=1,
#         tooltip=row['town'] + ': ' + row['location']
#     ).add_to(map)

# folium.Marker(
#                 location=[current_position['latitude'], current_position['longitude']],
#                 icon=folium.Icon(icon="car", prefix="fa")
#         ).add_to(map)

# map.options['maxBounds'] = bounds
# map.options['maxBoundsViscosity'] = 0.9
# map.options['minZoom'] = 5

# map


In [ ]:
# from plotter import Plotter

# plotter = Plotter(route)
# plotter.add_api_sites(api_requester.get_current_sites)
# plotter.add_control_stops(route.get_control_stops_data)
# # plotter.add_current_position(current_position)
# current_position = {'longitude': 132.213897,
#                     'latitude': -14.425286}
# plotter.update_max_speed_distance(current_position)
# plotter.plot

# #TODO 
# #SE SI CHIAMA CLOUDMOVE NON ELIMINARE ALTRI DATI METEO MA UNIRLI
# #SE NON CI SONO DATI PLOTTARE NO DATA

In [9]:
#TODO i dati vengono sovrascritti e non aggiornati nei plot
# tenere ultimo SF e aggiornare dati di CM dove possibile
# se non ci sono dati di SF usare CM e viceversa

# se codice runna per tanto, i dropdown del tempo devono essere aggiornati

# per i weather plot bisogna correggere il cumDistance cosi che da 0 fino al primo cumDistance i valori siano 0 per tutti i tempi
# lo stesso vale dall'ultimo punto fino a route_data['cumDistance'].max()

# cambiare i font

import pandas as pd
import dash
import constants
import plotly.graph_objects as go
from dash import Dash, html, dcc
from dash.dependencies import Input, Output, State
from functions import rad2deg


control_stops_df = route.get_control_stops_data
sites_df = api_requester.get_current_sites

app = Dash(__name__)

# aCe images
url_image = 'https://avatars.githubusercontent.com/u/115261983?s=280&v=4'

## Time
# Slider
height_slider = 1000
unique_times = preprocessed_df.index.get_level_values('time').unique()
slider_marks = {i: time.strftime('%H:%M') for i, time in enumerate(unique_times)}

# Dropdown
unique_days = unique_times.map(lambda x: x.strftime('%d %B %Y')).unique().to_list()

# Update time
update_sec = 50

## Map
route_data = route.get_route_data
max_cumDistance_km = route_data['cumDistance'].max() / 1000
row_current_cumDistance, _ = route.find_closest_row(current_position)
current_cumDistance_km = row_current_cumDistance['cumDistance'] / 1000
min_lng = route_data['longitude'].min()
min_lat = route_data['latitude'].min()
max_lng = route_data['longitude'].max()
max_lat = route_data['latitude'].max()
middle_lat = (max_lat + min_lat) / 2
middle_lng = (max_lng + min_lng) / 2

## Plot
x_cumDistance_km = route_data['cumDistance'] / 1000
x_cumDistance_str = 'Cumulative Distance (km)'

grid_color = 'lightgray'
box_color = 'black'

height_plot = 280

# Control stops, start, and finish lines
control_stops_df = route.get_control_stops_data

control_stops = control_stops_df.iloc[1:-1]
cs_vertical_lines = []
for _, row in control_stops.iterrows():
    x_value = row['cumDistance'] / 1000
    cs_vertical_lines.append(
        dict(type='line', x0=x_value, x1=x_value, y0=0, y1=1, yref='paper', line=dict(color='orange', dash='dash'), hovertext=row['town'] + ': ' + row['location'], hoverinfo='text')
    )
#TODO CHECK HOVERINFO
start_finish = control_stops_df.iloc[[0, -1]]
start_finish_lines = []
for _, row in start_finish.iterrows():
    x_value = row['cumDistance'] / 1000
    start_finish_lines.append(
        dict(type='line', x0=x_value, x1=x_value, y0=0, y1=1, yref='paper', line=dict(color='black', width=0.5), hovertext=row['town'] + ': ' + row['location'], hoverinfo='text')
    )
    
vertical_lines = cs_vertical_lines + start_finish_lines

## Function to initialize plots
def initialize_plot(x_list, y_list, title, vertical_lines, height_plot) -> go.Figure:
    return go.Figure(data=[go.Scatter(x=x_list, y=y_list, mode='lines', hoverinfo='none')],
                            layout=go.Layout(
                                margin={"r": 0, "t": 0},
                                xaxis={'title': 'x_cumDistance_str'},  # Update this as needed
                                yaxis={'title': title},
                                height=height_plot,
                                shapes=vertical_lines
                            ))


app.layout = html.Div([
    # Header with image and title
    html.Div([
        # Title
        html.H1("Strategy Interface",
                style={'textAlign': 'left', 'margin': '0 auto', 'flex': '1'}),

        # Dropdown
        dcc.Dropdown(options=[{'label': day, 'value': day} for day in unique_days],
                     value=unique_days[0],
                     id='day-month-year-dropdown',
                     style={'width': '200px', 'margin-right': '20px'}),
        
        # Last update
        dcc.Interval(id='interval-update', interval=update_sec*1000, n_intervals=0),
        html.Div([
            html.Div(id='last-update-sf',
                     style={'color': 'gray'}),
            html.Div(id='last-update-cm',
                     style={'color': 'gray'}),
            html.Div(id='last-update-sc',
                     style={'color': 'gray'}),
        ], style={'display': 'flex', 'flexDirection': 'column', 'margin-right': '20px'}),

        # Image
        html.Img(src=url_image,
                 style={'height': '100px', 'width': 'auto', 'margin-left': '20px'}),
        
    ], style={'display': 'flex', 'alignItems': 'center', 'padding-left': '20px', 'padding-right': '20px'}),


    # Container for the scouts
    html.Div([
        html.Div([
            dcc.Input(id='input-box-cumDist', type='number', min=current_cumDistance_km, max=max_cumDistance_km, placeholder='Distance from Darwin in km'),
            html.Button('Submit', id='submit-button-0', disabled=True),
            html.Div(id='output-container-0')
        ]),

        html.Div([
            dcc.Input(id='input-box-lat', type='number', min=constants.GEO['latitude']['min'], max=constants.GEO['latitude']['max'], placeholder='Latitude in °'),
            dcc.Input(id='input-box-lng', type='number', min=constants.GEO['longitude']['min'], max=constants.GEO['longitude']['max'], placeholder='Longitude in °'),
            html.Button('Submit', id='submit-button-1', disabled=True),
            html.Div(id='output-container-1')
        ]),
        html.Div([
            html.Table([
                # Header with merged cells
                html.Tr([html.Th('Current position', colSpan=2)]),
                
                # Remaining rows
                html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                html.Tr([html.Td(id='current_lat'), html.Td(id='current_lng')]),
                html.Tr([html.Td('Distance from Darwin'), html.Td('Distance to Adelaide')]),
                html.Tr([html.Td(id='dist-darwin'), html.Td(id='dist-ade')]),
            ]),
            html.Div([
                html.Table([
                    # Header with merged cells
                    html.Tr([html.Th('Next control stop', colSpan=2)]),
                    
                    # Remaining rows
                    html.Tr([html.Th(id='cs-name', colSpan=2)]),
                    html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
                    html.Tr([html.Td(id='cs_lat'), html.Td(id='cs_lng')]),
                    html.Tr([html.Td('Distance from Darwin'), html.Td('Distance from us')]),
                    html.Tr([html.Td(id='cs-dist-darwin'), html.Td(id='cs-dist-us')]),
                ])
            ])
        ])
    ], style={'display': 'flex', 'alignItems': 'center', 'padding-left': '20px', 'padding-right': '20px'}),


    # Container for the columns
    html.Div([
        # Map and static plots
        html.Div(children=[
            # Map
            dcc.Graph(id='map',
                config={'displayModeBar': 'hover'},
                figure={
                    'data': [{
                        'type': 'scattermapbox',
                        'lat': route_data['latitude'],
                        'lon': route_data['longitude'],
                        'mode': 'lines+markers',
                        'marker': {'size': 3},
                        'line': {'color': 'blue'},
                        'hoverinfo': 'none',
                        'name': 'Route'
                    },
                    {
                        'type': 'scattermapbox',
                        'lat': sites_df['latitude'],
                        'lon': sites_df['longitude'],
                        'mode': 'markers',
                        'marker': {'size': 3},
                        'line': {'color': 'white'},
                        'hoverinfo': 'none',
                        'name': 'Weather locations'
                    },
                    {
                        'type': 'scattermapbox',
                        'lat': control_stops_df['latitude'],
                        'lon': control_stops_df['longitude'],
                        'mode': 'markers',
                        'marker': {'size': 5},
                        'line': {'color': 'orange'},
                        'hovertext': (control_stops_df['town'] + ": " + control_stops_df['location']).to_list(),
                        'hoverinfo': 'text',
                        'name': 'Control Stops'
                    },
                    {
                        'type': 'scattermapbox',
                        'lat': [current_position['latitude']],
                        'lon': [current_position['longitude']],
                        'mode': 'markers',
                        'marker': {'size': 10, 'color': 'red'},
                        'name': 'Aletch'
                    }],
                    'layout': {
                        'mapbox': {
                            'style': "mapbox://styles/mapbox/satellite-streets-v11",
                            'accesstoken': constants.TOKEN_MAPBOX,
                            'center': {'lat': middle_lat, 'lon': middle_lng},
                            'zoom': 3.5,
                            'minzoom': 3.5
                        },
                        'margin': {'l': 0, 'r': 0, 'b': 0, 't': 0},
                        'legend': {
                            'x': 0,
                            'y': 1,
                            'xanchor': 'left',
                            'yanchor': 'top',
                            'bgcolor': 'transparent'
                        }
                    }
                }
            ),

            # Velocity
            html.Br(),
            dcc.Graph(id='velocity',
                figure={
                    'data': [
                        {
                            'x': x_cumDistance_km,
                            'y': route_data['maxSpeed'],
                            'type': 'scatter',
                            'mode': 'lines',
                            'name': 'Max speed',
                            'hoverinfo': 'none',
                        },
                        {
                            'x': DP_data['cumDist_DP'] / 1000,
                            'y': DP_data['optV_DP'],
                            'type': 'scatter',
                            'mode': 'lines',
                            'name': 'DP long-term',
                            'hoverinfo': 'none',
                        }
                    ],
                    'layout': {
                        'margin': {
                            "r": 0,
                            "t": 0
                            },
                        'xaxis': {
                            'title': x_cumDistance_str,
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'yaxis': {
                            'title': 'Velocity (km/h)',
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'legend': {
                        'x': 1,
                        'y': 1,
                        'xanchor': 'right',
                        'yanchor': 'top'},
                        'shapes': vertical_lines,
                        'height': height_plot
                    }
                }
            ),

            # SoC
            html.Br(),
            dcc.Graph(id='soc',
                figure={
                    'data': [
                        {
                            'x': DP_data['cumDist_DP'] / 1000,
                            'y': DP_data['optSoC_DP'],
                            'type': 'scatter',
                            'mode': 'lines',
                            'name': 'DP long-term',
                            'hoverinfo': 'none',
                        }
                    ],
                    'layout': {
                        'margin': {
                            "r": 0,
                            "t": 0
                            },
                        'xaxis': {
                            'title': x_cumDistance_str,
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'yaxis': {
                            'title': 'SoC (%)',
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        # 'legend': {
                        # 'x': 1,
                        # 'y': 1,
                        # 'xanchor': 'right',
                        # 'yanchor': 'top'},
                        'shapes': vertical_lines,
                        'height': height_plot
                    }
                }
            ),

            # Altitude
            html.Br(),
            dcc.Graph(id='altitude',
                figure={
                    'data': [
                        {
                            'x': x_cumDistance_km,
                            'y': route_data['altitudeSmoothed'],
                            'type': 'scatter',
                            'mode': 'lines',
                            'hoverinfo': 'none',
                        },
                    ],
                    'layout': {
                        'margin': {
                            "r": 0,
                            "t": 0
                            },
                        'xaxis': {
                            'title': x_cumDistance_str,
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'yaxis': {
                            'title': 'Altitude (m)',
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'shapes': vertical_lines,
                        'height': height_plot
                    }
                }
            ),

            html.Br(),
            dcc.Graph(id='inclination',
                figure={
                    'data': [
                        {
                            'x': x_cumDistance_km,
                            'y': route_data['inclinationSmoothed'].apply(rad2deg),
                            'type': 'scatter',
                            'mode': 'lines',
                            'hoverinfo': 'none',
                        },
                    ],
                    'layout': {
                        'margin': {
                            "r": 0,
                            "t": 0
                            },
                        'xaxis': {
                            'title': x_cumDistance_str,
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'yaxis': {
                            'title': 'Inclination (°)',
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color
                        },
                        'shapes': vertical_lines,
                        'height': height_plot
                    }
                }
            )
        ], style={'padding': 10, 'flex': 1}),

        # Time slider
        html.Div(
            dcc.RangeSlider(
                id='hour-minute-range-slider',
                min=0,
                max=len(unique_days) - 1,
                value=[0, len(unique_days) - 1],
                marks=slider_marks,
                vertical=True,
                step=1,
                verticalHeight=height_slider
            ),
            style={'width': '70px', 'height':height_slider},
            id='hour-minute-range-slider-container'
        ),

        # Weather plots
        html.Div(children=[
            dcc.Graph(id='plot-globalIrradiance',
                figure={
                    'data': [{
                        'x': x_cumDistance_km,
                        'y': x_cumDistance_km,
                        'type': 'scatter',
                        'mode': 'lines',
                        'hoverinfo': 'none'}],
                    'layout': {
                        'margin': {"r": 0,
                                "t": 0},
                        'xaxis': {
                            'title': x_cumDistance_str,
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color},
                        'yaxis': {
                            'title': 'Global irradiance (W/m²)',
                            'showgrid': True,
                            'gridcolor': grid_color,
                            'showline': True,
                            'linecolor': box_color},
                        'shapes': vertical_lines,
                        'height': height_plot
                    }
                }
            ),
            dcc.Graph(id='plot-temperature'),
            dcc.Graph(id='plot-wind'),
            dcc.Graph(id='plot-precipitation'),

        ], style={'padding': 10, 'flex': 1})
    ], style={'display': 'flex', 'flex-direction': 'row'})
    
], style={'display': 'flex', 'flex-direction': 'column'})

### Callbacks
# Last update
@app.callback(
    Output('last-update-sf', 'children'),
    Output('last-update-cm', 'children'),
    Output('last-update-sc', 'children'),
    Input('interval-update', 'n_intervals')
)
def update_output_div(n):
    time_SF_str = api_requester.get_solar_forecast_last_time
    if time_SF_str is pd.NaT:
        print_SF = "Last update SF: No data available"
    else:
        print_SF = f"Last update SF: {time_SF_str.strftime('%d %B %Y at %H:%M')}"

    time_CM_str = api_requester.get_solar_forecast_cloudmove_last_time
    if time_CM_str is pd.NaT:
        print_CM = "Last update CM: No data available"
    else:
        print_CM = f"Last update CM: {time_CM_str.strftime('%d %B %Y at %H:%M')}"

    time_SC_str = solcast_executer.get_solcast_last_time
    if time_SC_str is pd.NaT:
        print_SC = "Last update SC: No data available"
    else:
        print_SC = f"Last update SC: {time_SC_str.strftime('%d %B %Y at %H:%M')}"

    return print_SF, print_CM, print_SC

# Time callback
@app.callback(
    [Output('hour-minute-range-slider', 'max'),
     Output('hour-minute-range-slider', 'marks'),
     Output('hour-minute-range-slider', 'value'),
     Output('hour-minute-range-slider-container', 'style')],
     Input('day-month-year-dropdown', 'value')
)
def update_slider(day_month_year):
    min_value = 0
    now = pd.Timestamp.now(tz=constants.TIMEZONE)
    
    unique_times_days = unique_times[unique_times.map(lambda x: x.strftime('%d %B %Y')) == day_month_year]
    slider_marks = {i: {'label': time.strftime('%H:%M')} for i, time in enumerate(unique_times_days)}

    next_time_index = unique_times_days.searchsorted(now)
    if next_time_index < len(unique_times_days) and now.day == pd.to_datetime(day_month_year).day:
        slider_marks[next_time_index]['style'] = {'color': 'red'}
        min_value = next_time_index

    # If 17 is present, update max_value to that value
    #TODO
    # Otherwise
    max_value = len(unique_times_days) - 1
    value = [min_value, max_value]
    style = {'width': '100px', 'height': f'{max_value*60}px'}

    return max_value, slider_marks, value, style

# Axis callback
@app.callback(
    [Output('velocity', 'figure'),
     Output('soc', 'figure'),
     Output('altitude', 'figure'),
     Output('inclination', 'figure'),
     Output('plot-globalIrradiance', 'figure', allow_duplicate=True),
     Output('plot-temperature', 'figure', allow_duplicate=True),
     Output('plot-wind', 'figure', allow_duplicate=True),
     Output('plot-precipitation', 'figure', allow_duplicate=True)],
    [Input('velocity', 'relayoutData'),
     Input('soc', 'relayoutData'),
     Input('altitude', 'relayoutData'),
     Input('inclination', 'relayoutData'),
     Input('plot-globalIrradiance', 'relayoutData'),
     Input('plot-temperature', 'relayoutData'),
     Input('plot-wind', 'relayoutData'),
     Input('plot-precipitation', 'relayoutData')],
    [State('velocity', 'figure'),
     State('soc', 'figure'),
     State('altitude', 'figure'),
     State('inclination', 'figure'),
     State('plot-globalIrradiance', 'figure'),
     State('plot-temperature', 'figure'),
     State('plot-wind', 'figure'),
     State('plot-precipitation', 'figure')],
    prevent_initial_call=True)
def update_xaxis(v, soc, h, dh, gh, T, w, rr, fig_v, fig_soc, fig_h, fig_dh, fig_gh, fig_T, fig_w, fig_rr):
    # Get the callback context to determine which input triggered the callback
    ctx = dash.callback_context
    if not ctx.triggered:
        raise dash.exceptions.PreventUpdate
    input_id = ctx.triggered[0]['prop_id'].split('.')[0]

    trigger_value = ctx.triggered[0]['value']

    # Check if autorange is set for any of the graphs
    if 'autosize' in trigger_value and trigger_value['autosize']:
        if trigger_value['autosize']:
            raise dash.exceptions.PreventUpdate
        
    # PreventUpdate when changing yxis only
    elif 'yaxis.range[0]' in trigger_value or 'yaxis.range[1]' in trigger_value:
        raise dash.exceptions.PreventUpdate
    
    elif 'xaxis.autorange' in trigger_value and trigger_value['xaxis.autorange']:
        # If autorange is set for any graph, let all graphs determine their x-axis range automatically
        fig_v['layout']['xaxis']['autorange'] = True
        fig_soc['layout']['xaxis']['autorange'] = True
        fig_h['layout']['xaxis']['autorange'] = True
        fig_dh['layout']['xaxis']['autorange'] = True
        fig_gh['layout']['xaxis']['autorange'] = True
        fig_T['layout']['xaxis']['autorange'] = True
        fig_w['layout']['xaxis']['autorange'] = True
        fig_rr['layout']['xaxis']['autorange'] = True

    else:
        # Determine the new x-axis range
        if input_id == 'velocity':
            xaxis_range = [v['xaxis.range[0]'], v['xaxis.range[1]']]
        elif input_id == 'soc':
            xaxis_range = [soc['xaxis.range[0]'], soc['xaxis.range[1]']]
        elif input_id == 'altitude':
            xaxis_range = [h['xaxis.range[0]'], h['xaxis.range[1]']]
        elif input_id == 'inclination':
            xaxis_range = [dh['xaxis.range[0]'], dh['xaxis.range[1]']]
        elif input_id == 'plot-globalIrradiance':
            xaxis_range = [gh['xaxis.range[0]'], gh['xaxis.range[1]']]
        elif input_id == 'plot-temperature':
            xaxis_range = [T['xaxis.range[0]'], T['xaxis.range[1]']]
        elif input_id == 'plot-wind':
            xaxis_range = [w['xaxis.range[0]'], w['xaxis.range[1]']]
        elif input_id == 'plot-precipitation':
            xaxis_range = [rr['xaxis.range[0]'], rr['xaxis.range[1]']]
        else:
            raise dash.exceptions.PreventUpdate

        # Update the x-axis range for all figures and set autorange to False
        fig_v['layout']['xaxis']['range'] = xaxis_range
        fig_v['layout']['xaxis']['autorange'] = False

        fig_soc['layout']['xaxis']['range'] = xaxis_range
        fig_soc['layout']['xaxis']['autorange'] = False

        fig_h['layout']['xaxis']['range'] = xaxis_range
        fig_h['layout']['xaxis']['autorange'] = False

        fig_dh['layout']['xaxis']['range'] = xaxis_range
        fig_dh['layout']['xaxis']['autorange'] = False

        fig_gh['layout']['xaxis']['range'] = xaxis_range
        fig_gh['layout']['xaxis']['autorange'] = False

        fig_T['layout']['xaxis']['range'] = xaxis_range
        fig_T['layout']['xaxis']['autorange'] = False

        fig_w['layout']['xaxis']['range'] = xaxis_range
        fig_w['layout']['xaxis']['autorange'] = False

        fig_rr['layout']['xaxis']['range'] = xaxis_range
        fig_rr['layout']['xaxis']['autorange'] = False

    return fig_v, fig_soc, fig_h, fig_dh, fig_gh, fig_T, fig_w, fig_rr

####TODO CAMBIO TEMPO NON SI RIPLOTTA TUTTO LA FIGURE, MA SI AGGIORNANO SOLO DATI CON AXIS VECCHIE
# Weather plots
@app.callback(
    [Output('plot-globalIrradiance', 'figure'),
     Output('plot-temperature', 'figure'),
     Output('plot-wind', 'figure'),
     Output('plot-precipitation', 'figure')],
    [Input('day-month-year-dropdown', 'value'),
    Input('hour-minute-range-slider', 'value'),
    Input('hour-minute-range-slider', 'marks'),
    ],
    State('plot-globalIrradiance', 'figure')
    )
def update_graphs(day_month_year, hour_minute, slider_marks, figure_gh): #
    # Extract timezone from the DataFrame's index (should be constants.TIMEZONE)
    timezone = preprocessed_df.index.get_level_values('time').tz 

    # Create the start and end timestamps for the day
    start_time = pd.Timestamp(day_month_year).tz_localize(timezone)
    end_time = start_time + pd.Timedelta(days=1)

    # Filter the DataFrame for the specific day
    data_for_selected_day = preprocessed_df[(preprocessed_df.index.get_level_values('time') >= start_time) & 
                            (preprocessed_df.index.get_level_values('time') < end_time)]
    
    # Create the start and end timestamps for the range
    first_datetime_str = day_month_year + " " + slider_marks[f'{hour_minute[0]}']['label'] # Slider marks are str and hour_minute int
    first_datetime_obj = pd.Timestamp(first_datetime_str).tz_localize(timezone)
    last_datetime_str = day_month_year + " " + slider_marks[f'{hour_minute[1]}']['label'] # Slider marks are str and hour_minute int
    last_datetime_obj = pd.Timestamp(last_datetime_str).tz_localize(timezone)

    # Create a mask to select the data in the range
    mask = (data_for_selected_day.index.get_level_values('time') >= first_datetime_obj) & (data_for_selected_day.index.get_level_values('time') <= last_datetime_obj)
    data_in_range_df = data_for_selected_day[mask]

    x_cumDistance_range_km = data_in_range_df.index.get_level_values('cumDistance').unique() / 1000

    figure_gh['data'][0]['x'] = x_cumDistance_range_km
    figure_gh['data'][0]['y'] = data_in_range_df['globalIrradiance'].groupby(level='cumDistance').mean()
    line_shape = [{
                'type': 'line',
                'x0': x_cumDistance_range_km.min(),
                'x1': x_cumDistance_range_km.max(),
                'y0': data_for_selected_day['globalIrradiance'].max(),
                'y1': data_for_selected_day['globalIrradiance'].max(),
                'line': {
                    'color': 'red',
                    'width': 1}
                },
            ]
    figure_gh['layout']['shapes'] = vertical_lines + line_shape

    print(figure_gh['layout']['shapes'])


    # figure_gh={
    #     'data': [{
    #         'x': x_cumDistance_range_km,
    #         'y': data_in_range_df['globalIrradiance'].groupby(level='cumDistance').mean(),
    #         'type': 'scatter',
    #         'mode': 'lines',
    #         'hoverinfo': 'none'}],
    #     'layout': {
    #         'margin': {"r": 0,
    #                    "t": 0},
    #         'xaxis': {
    #             'title': x_cumDistance_str,
    #             'showgrid': True,
    #             'gridcolor': grid_color,
    #             'showline': True,
    #             'linecolor': box_color},
    #         'yaxis': {
    #             'title': 'Global irradiance (W/m²)',
    #             'showgrid': True,
    #             'gridcolor': grid_color,
    #             'showline': True,
    #             'linecolor': box_color},
    #         'shapes': [{
    #             'type': 'line',
    #             'x0': x_cumDistance_range_km.min(),
    #             'x1': x_cumDistance_range_km.max(),
    #             'y0': data_for_selected_day['globalIrradiance'].max(),
    #             'y1': data_for_selected_day['globalIrradiance'].max(),
    #             'line': {
    #                 'color': 'red',
    #                 'width': 1}
    #             },
    #         ] + vertical_lines,
    #         'height': height_plot
    #     }
    # }

    figure_tt={
        'data': [{
            'x': x_cumDistance_range_km,
            'y': data_in_range_df['temperature'].groupby(level='cumDistance').mean(),
            'type': 'scatter',
            'mode': 'lines',
            'hoverinfo': 'none'}],
        'layout': {
            'margin': {"r": 0,
                       "t": 0},
            'xaxis': {
                'title': x_cumDistance_str,
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'yaxis': {
                'title': 'Temperature (°C)',
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'shapes': [{
                'type': 'line',
                'x0': x_cumDistance_range_km.min(),
                'x1': x_cumDistance_range_km.max(),
                'y0': data_for_selected_day['temperature'].max(),
                'y1': data_for_selected_day['temperature'].max(),
                'line': {
                    'color': 'red',
                    'width': 1}
                },
                {
                'type': 'line',
                'x0': x_cumDistance_range_km.min(),
                'x1': x_cumDistance_range_km.max(),
                'y0': data_for_selected_day['temperature'].min(),
                'y1': data_for_selected_day['temperature'].min(),
                'line': {
                    'color': 'blue',
                    'width': 1}
                }
            ] + vertical_lines,
            'height': height_plot
        }
    }

    figure_ww={
        'data': [{
            # 'x': x_cumDistance_range_km,
            # 'y': data_in_range_df['windGust'].groupby(level='cumDistance').mean(),
            # 'type': 'scatter',
            # 'mode': 'lines',
            # 'name': 'Wind gust'},
            # {
            'x': x_cumDistance_range_km,
            'y': data_in_range_df['windSpeed'].groupby(level='cumDistance').mean(),
            'type': 'scatter',
            'mode': 'lines',
            'name': 'Wind speed',
            'hoverinfo': 'none'},
            {
            'x': x_cumDistance_range_km,
            'y': abs(data_in_range_df['sideWind'].groupby(level='cumDistance').mean()),
            'type': 'scatter',
            'mode': 'lines',
            'name': 'Side wind',
            'hoverinfo': 'none'},
            {
            'x': x_cumDistance_range_km,
            'y': data_in_range_df['frontWind'].groupby(level='cumDistance').mean(),
            'type': 'scatter',
            'mode': 'lines',
            'name': 'Front wind',
            'hoverinfo': 'none',},
            ],
        'layout': {
            'margin': {"r": 0,
                       "t": 0},
            'xaxis': {
                'title': x_cumDistance_str,
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'yaxis': {
                'title': 'Wind (km/h)',
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'height': height_plot,
            'legend': {
                'x': 1,
                'y': 1,
                'xanchor': 'right',
                'yanchor': 'top'},
            'shapes': vertical_lines
        }
    }

    figure_rr={
        'data': [{
            'x': x_cumDistance_range_km,
            'y': round(data_in_range_df['precipitation'].groupby(level='cumDistance').mean(), 2),
            'type': 'scatter',
            'mode': 'lines',
            'hoverinfo': 'none',}],
        'layout': {
            'margin': {"r": 0,
                       "t": 0},
            'xaxis': {
                'title': x_cumDistance_str,
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'yaxis': {
                'title': 'Precipitation (mm)',
                'showgrid': True,
                'gridcolor': grid_color,
                'showline': True,
                'linecolor': box_color},
            'shapes': vertical_lines,
            'height': height_plot,
            'legend': {
                'x': 0.2,
                'y': 1,
                'xanchor': 'left',
                'yanchor': 'top'
            }
        }
    }

    return figure_gh, figure_tt, figure_ww, figure_rr

## Callback Scouts
@app.callback(
    Output('submit-button-0', 'disabled'),
    Input('input-box-cumDist', 'value')
)
def update_button_cumDistance(input_cumDistance_km):
    # If the input value is not None and is within the range, enable the button
    if input_cumDistance_km is not None and current_cumDistance_km <= input_cumDistance_km <= max_cumDistance_km:
        return False
    return True

@app.callback(
    Output('output-container-0', 'children'),
    Input('submit-button-0', 'n_clicks'),
    State('input-box-cumDist', 'value')
)
def update_output_cumDistance(n_clicks, input_cumDistance_km):
    if n_clicks:
        # If smaller than current cumDistance
        if input_cumDistance_km < current_cumDistance_km:
            return 'Error: position before current position'
        
        position_series, _ = route.find_closest_row_cumDistance(input_cumDistance_km * 1000)
        lat = position_series['latitude']
        lng = position_series['longitude']
        
        # Calculate the time difference between now and 17:00 of the current day
        now = pd.Timestamp.now()
        target_time = now.replace(hour=17, minute=0, second=0, microsecond=0)
        delta_time = (target_time - now).seconds / 3600

        # Check if the current time is already past 17:00
        if delta_time < 0:
            return 'Error: The current time is past 17:00'
        
        mean_v = (input_cumDistance_km - current_cumDistance_km) / delta_time
        return f'{lat}, {lng} Mean velocity: {round(mean_v, 1)} km/h'

@app.callback(
    Output('submit-button-1', 'disabled'),
    Input('input-box-lat', 'value'),
    Input('input-box-lng', 'value')
)
def update_button_latlon(input_lat, input_lng):
    # Check if both values are in the specified range
    if input_lat is not None and constants.GEO['latitude']['min'] <= input_lat <= constants.GEO['latitude']['max'] and input_lng is not None and constants.GEO['longitude']['min'] <= input_lng <= constants.GEO['longitude']['max']:
        return False
    return True

@app.callback(
    Output('output-container-1', 'children'),
    Input('submit-button-1', 'n_clicks'),
    State('input-box-lat', 'value'),
    State('input-box-lng', 'value')
)
def update_output_latlon(n_clicks, input_lat, input_lng):
    if n_clicks:
        if input_lat is None or not (constants.GEO['latitude']['min'] <= input_lat <= constants.GEO['latitude']['max']) or input_lng is None or not (constants.GEO['longitude']['min'] <= input_lng <= constants.GEO['longitude']['max']):
            return 'Error: One or both values are out of range!'
        
        position = {'latitude': input_lat, 'longitude': input_lng}
        position_series, _ = route.find_closest_row(position)
        position_cumDistance = position_series['cumDistance']

        # If smaller than current cumDistance
        if position_cumDistance < (current_cumDistance_km * 1000):
            return 'Error: position before current position'
        
        # Calculate the time difference between now and 17:00 of the current day
        now = pd.Timestamp.now()
        target_time = now.replace(hour=17, minute=0, second=0, microsecond=0)
        delta_time = (target_time - now).seconds / 3600

        # Check if the current time is already past 17:00
        if delta_time < 0:
            return 'Error: The current time is past 17:00'
        
        mean_v = (position_cumDistance / 1000 - current_cumDistance_km) / delta_time
        return f'{round(position_cumDistance / 1000, 2)} Mean velocity: {round(mean_v, 1)} km/h'

# Current position
@app.callback(
    [Output('current_lat', 'children'),
     Output('current_lng', 'children'),
     Output('dist-darwin', 'children'),
     Output('dist-ade', 'children')],
    Input('interval-update', 'n_intervals')
)
def update_current_position(n_intervals):
    # Get the current position
    # current_position = route.get_current_position() #TODO CHANGE WITH GPS
    row, _ = route.find_closest_row(current_position)
    return current_position['latitude'], current_position['longitude'], round(row['cumDistance'] / 1000, 1), round(max_cumDistance_km - (row['cumDistance'] / 1000), 1)

# Control stops
@app.callback(
    [Output('cs_lat', 'children'),
     Output('cs_lng', 'children'),
     Output('cs-dist-darwin', 'children'),
     Output('cs-dist-us', 'children'),
     Output('cs-name', 'children')],
    Input('interval-update', 'n_intervals')
)
def next_control_stop(n_intervals):
    # Get the current position
    # current_position = route.get_current_position() #TODO CHANGE WITH GPS
    row_route, _ = route.find_closest_row(current_position)
    current_cum_dist_km = row_route['cumDistance'] / 1000

    row, _ = route.find_next_cs(current_position, print_is_requested=False)
    cs_cum_distance_km = row['cumDistance'] / 1000
    name = row['town'] + ': ' + row['location']

    delta_dis = cs_cum_distance_km - current_cum_dist_km

    return row['latitude'], row['longitude'], round(cs_cum_distance_km, 1), round(delta_dis, 1), name

if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="tab")


Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

[{'type': 'line', 'x0': 323.04640659192, 'x1': 323.04640659192, 'y0': 0, 'y1': 1, 'yref': 'paper', 'line': {'color': 'orange', 'dash': 'dash'}, 'hovertext': 'Katherine: Transport Weighbridge', 'hoverinfo': 'text'}, {'type': 'line', 'x0': 634.104056068814, 'x1': 634.104056068814, 'y0': 0, 'y1': 1, 'yref': 'paper', 'line': {'color': 'orange', 'dash': 'dash'}, 'hovertext': 'Dunmarra:  Dunmarra Wayside Inn', 'hoverinfo': 'text'}, {'type': 'line', 'x0': 990.065645214209, 'x1': 990.065645214209, 'y0': 0, 'y1': 1, 'yref': 'paper', 'line': {'color': 'orange', 'dash': 'dash'}, 'hovertext': 'Tennant Creek: Tennant Creek Weighbridge', 'hoverinfo': 'text'}, {'type': 'line', 'x0': 1213.58013387552, 'x1': 1213.58013387552, 'y0': 0, 'y1': 1, 'yref': 'paper', 'line': {'color': 'orange', 'dash': 'dash'}, 'hovertext': 'Barrow Creek: Barrow Creek Hotel', 'hoverinfo': 'text'}, {'type': 'line', 'x0': 1497.79759471453, 'x1': 1497.79759471453, 'y0': 0, 'y1': 1, 'yref': 'paper', 'line': {'color': 'orange', 'd

In [ ]:
# # Weather plots

# app.layout = html.Div(
#     dcc.Graph(id='plot-globalIrradiance',
#               figure_gh={
#                 'data': [{
#                     'x': [],
#                     'y': [],
#                     'type': 'scatter',
#                     'mode': 'lines',
#                     'hoverinfo': 'none'}],
#                 'layout': {
#                     'margin': {"r": 0,
#                             "t": 0},
#                     'xaxis': {
#                         'title': x_cumDistance_str,
#                         'showgrid': True,
#                         'gridcolor': grid_color,
#                         'showline': True,
#                         'linecolor': box_color},
#                     'yaxis': {
#                         'title': 'Global irradiance (W/m²)',
#                         'showgrid': True,
#                         'gridcolor': grid_color,
#                         'showline': True,
#                         'linecolor': box_color},
#                     'shapes': vertical_lines,
#                     'height': height_plot
#                 }
#             }
#         ),
#     html.Button(id='id')
# )

# @app.callback(
#     Output('plot-globalIrradiance', 'figure'),
#     Input('button', 'disable'),
#     Input('plot-globalIrradiance', 'figure')
#     )
# def update_graphs(button_disable, figure_gh):
#     x_cumDistance_range_km = data_in_range_df.index.get_level_values('cumDistance').unique() / 1000

#     figure_gh['data'][0]['x'] = x_cumDistance_range_km
#     figure_gh['data'][0]['y'] = data_in_range_df['globalIrradiance'].groupby(level='cumDistance').mean()
#     figure_gh['shapes'] = [{
#                 'type': 'line',
#                 'x0': x_cumDistance_range_km.min(),
#                 'x1': x_cumDistance_range_km.max(),
#                 'y0': data_for_selected_day['globalIrradiance'].max(),
#                 'y1': data_for_selected_day['globalIrradiance'].max(),
#                 'line': {
#                     'color': 'red',
#                     'width': 1}
#                 },
#             ]

#     return figure_gh


In [ ]:
def _recursive_position_finder(current_cumDistance:float, driving_time:float, cs_to_skip:int, i:int) -> pd.Series:
    """ """
    print(i)
    # Cut data at current position (lower cut)
    cut_data = route_data.copy()

    cut_data = cut_data[cut_data['cumDistance'] >= current_cumDistance]
    cut_data = cut_data.reset_index(drop=True)

    current_time = cut_data['cumTimeAtMaxSpeedLim'][0]

    # Cut data at driving time (upper cut)
    cut_data = cut_data[cut_data['cumTimeAtMaxSpeedLim'] <= current_time + driving_time]
    max_cumDistance = cut_data['cumDistance'].max()

    # Check if the control stop dataframe is not empty
    if not control_stops.empty:
        cs_in_range_mask = (control_stops['cumDistance'] >= current_cumDistance) & (control_stops['cumDistance'] <= max_cumDistance)
        cs_in_range = cs_in_range_mask.sum()
        print(f'cs found ahead: {cs_in_range}')
        print(f'cs to skip: {cs_to_skip}')
    else:
        print("No control stop dataframe given")

    # Stop cases
    # Reach end of route, return last point
    if current_cumDistance >= route_data.iloc[-1]['cumDistance']:
        return route_data.iloc[-1] # return self.end_position
    
    # All control stops considered
    if cs_to_skip == cs_in_range:
        print("All control stops considered")
        return route_data.loc[route_data['cumDistance'] == max_cumDistance].iloc[0]
    
    # Stop at control stop for the night, meaning we arrive at cs between 16:30 and 17:00
    if cs_to_skip > cs_in_range:
        print("Stop at control stop for the night")
        return control_stops.loc[control_stops['cumDistance'] > current_cumDistance].iloc[cs_to_skip - 1]
    

    # Recursive call to skip control stop and reduce driving time by 30 minutes
    if cs_to_skip < cs_in_range: # Case of 0 cs in range considered
        print("--- Recursive call ---")
        return _recursive_position_finder(current_cumDistance, driving_time - 30.0*60.0, cs_to_skip + 1, i+1)
    
def update_max_speed_distance(current_position:dict) -> float:
    """ """
    # Subtract overnight stop start time to now
    now = pd.Timestamp.now()
    driving_time = pd.Timedelta(hours=17) - pd.Timedelta(hours=now.hour, minutes=now.minute)
    current_cumDistance = route.find_closest_row(current_position, print_is_requested=False)[0]['cumDistance']

    position_series = _recursive_position_finder(current_cumDistance, driving_time.total_seconds(), cs_to_skip=0, i=0)

    return position_series['cumDistance']

# from plotter import Plotter

# plotter = Plotter(route)
# # plotter.add_api_sites(api_requester.get_current_sites)
# # plotter.add_control_stops(route.get_control_stops_data)
# # plotter.add_current_position(current_position)
# # current_position = {'longitude': 132.213897,
# #                     'latitude': -14.425286}
# plotter_cum = plotter.update_max_speed_distance(current_position)

app = Dash(__name__)

app.layout = html.Div([
    dcc.Interval(id='interval-update', interval=update_sec*1000, n_intervals=0),
    html.Table([
        # Header with merged cells
        html.Tr([html.Th('Next overnight stop', colSpan=2)]),
        
        # Remaining rows
        html.Tr([html.Th('At max speed', colSpan=2)]),
        html.Tr([html.Td('Latitude'), html.Td('Longitude')]),
        html.Tr([html.Td(id='os_max_v_lat'), html.Td(id='os_max_v_lng')]),
        html.Tr([html.Td('Distance from Darwin'), html.Td('Distance from us')]),
        html.Tr([html.Td(id='os-max-v-dist-darwin'), html.Td(id='os-max-v-dist-us')]),
    ])
])

@app.callback(
    [Output('os_max_v_lat', 'children'),
     Output('os_max_v_lng', 'children'),
     Output('os-max-v-dist-darwin', 'children'),
     Output('os-max-v-dist-us', 'children')],
    Input('interval-update', 'n_intervals')
)
def next_overnight_stop(n_intervals):
    # Get the current position
    # current_position = route.get_current_position() #TODO CHANGE WITH GPS

    row_route, _ = route.find_closest_row(current_position)
    current_cum_dist_km = row_route['cumDistance'] / 1000
    print(current_cum_dist_km)

    os_cum_dist = update_max_speed_distance(current_position)
    row, _ = route.find_closest_row_cumDistance(os_cum_dist)
    os_cum_dist_km = os_cum_dist / 1000
    os_cum_dist_km2 = row['cumDistance'] / 1000
    print(os_cum_dist_km, os_cum_dist_km2)

    delta_dis = os_cum_dist_km - current_cum_dist_km

    return row['latitude'], row['longitude'], round(os_cum_dist_km, 1), round(delta_dis, 1)

if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode="tab")


Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

323.324706408675
0
cs found ahead: 0
cs to skip: 0
All control stops considered
447.94415164180896 447.94415164180896
323.324706408675
0
cs found ahead: 0
cs to skip: 0
All control stops considered
447.060459943525 447.060459943525
